In [1]:
from glob import glob
folder = glob("cabai/*/", recursive = True)

In [2]:
def filter_ini_dong(l1, folder):
    l2set = {x for x in l1}
    l3 = [x for x in folder if x not in l2set]
    return l3

l1 = [""]#cabai\\Antaraknosa\\
l3 = filter_ini_dong(l1, folder)
print(l3)

['cabai\\Antaraknosa\\', 'cabai\\Bercak Daun\\']


In [3]:
kumpulan_folder = []
for i in range(len(l3)):
    folder = glob(f"{l3[i]}/*/", recursive = True)
    kumpulan_folder.append(folder)
    
import itertools
kumpulan_folder = list(itertools.chain(*kumpulan_folder))
kumpulan_folder = [x for x in kumpulan_folder if not x.endswith('- TL\\')]
print(kumpulan_folder)
print(len(kumpulan_folder))

['cabai\\Antaraknosa\\Cabai - Antraknosa - 1 - 2022 12 20\\', 'cabai\\Antaraknosa\\Cabai - Antraknosa - 2 - 2022 12 20\\', 'cabai\\Antaraknosa\\Cabai - Antraknosa - 3 - 2022 12 20\\', 'cabai\\Antaraknosa\\Cabai - Antraknosa - 4 - 2022 12 20\\', 'cabai\\Bercak Daun\\Cabai - Bercak Daun - 1 - 2022 12 20\\', 'cabai\\Bercak Daun\\Cabai - Bercak Daun - 2 - 2022 12 20\\']
6


In [4]:
import xml.etree.ElementTree as ET
import glob
import os
import json
import shutil, os


def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


classes = []
output_dir_labels = "file_train/"
# create the labels folder (output directory)
os.mkdir(output_dir_labels)

# identify all the xml files in the annotations folder (input directory)
files = glob.glob(os.path.join(kumpulan_folder[0], '*.xml'))
# loop through each 
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    img_dirname = os.path.join(kumpulan_folder[0], f"{filename}.jpg")
    # check if the label contains the corresponding image file
    if not os.path.exists(os.path.join(kumpulan_folder[0], f"{filename}.jpg")):
        print(f"{filename} image does not exist!")
    else :
        result = []
        result_name = []
        #print(os.path.join(kumpulan_folder[0], f"{filename}.jpg"))
            # parse the content of the xml file

        result_name.append(img_dirname)

        tree = ET.parse(fil)
        root = tree.getroot()
        width = int(root.find("size").find("width").text)
        height = int(root.find("size").find("height").text)

        for obj in root.findall('object'):
            label = obj.find("name").text
            # label = "daun"
            # # check for new classes and append to list
            # if label=="kuning_keriting_daun_cabai":
            #     label="kuning_keriting"
            if label not in classes:
                classes.append(label)
            index = classes.index(label)
            pil_bbox = [int(x.text) for x in obj.find("bndbox")]
            yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
            # convert data to string
            bbox_string = " ".join([str(x) for x in yolo_bbox])
            result.append(f"{index} {bbox_string}")

        if result:
            # generate a YOLO format text file for each xml file
            with open(os.path.join(output_dir_labels, f"{filename}.txt"), "w", encoding="utf-8") as f:
                f.write("\n".join(result))
            
            for f in result_name:
                shutil.copy(img_dirname, output_dir_labels)

# generate the classes file as reference
with open('classes.txt', 'w', encoding='utf8') as f:
    f.write(json.dumps(classes))